In [1]:
import pandas as pd
import numpy as np
from utils.nucStitch import fullNucImage16
import os
from os.path import isfile, join
from skimage.measure import regionprops_table
from utils.nucStitch import fullNucImage16
from os import listdir
from inducedRainbow.spatial.kNNDistance import kNN

In [9]:
nucPath = "/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/nuclei"

In [10]:
folders = [x[0] for x in os.walk(nucPath)]
folders = sorted(folders)

In [57]:
dfNucDistancs = pd.DataFrame()
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    if files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        nucImage = fullNucImage16(folder)
        prop_dict = regionprops_table(nucImage, properties = ('label', 'centroid'))
        dfNuc = pd.DataFrame(prop_dict)
        nbrsNuc, distancesNuc, indicesNuc = kNN(dfNuc.iloc[:,1:3], 6)
        dfHelp = pd.DataFrame(distancesNuc)
        dfHelp["age"] = age
        dfHelp["aorta"] = aorta
        dfNucDistancs = pd.concat([dfNucDistancs, dfHelp])

In [82]:
dfNucDistancs.loc[(dfNucDistancs["age"] == "10-21") & (dfNucDistancs["aorta"] == "aorta01")][1].median()

7.365613110704396

In [94]:
savePath = "/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/nearestNeigbourNucleiComparison"
dfNucDistancs.to_csv(join(savePath, "nucleiNN.csv"), index=False)

In [4]:
#Nearest neighbour marked cells, not cellPose

In [26]:
nucleiPath = '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/nuclei'
clusterPath = '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red'
savePath = "/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/nearestNeigbourNucleiComparison/marked"

In [29]:
clusterFolders = [x[0] for x in os.walk(clusterPath)]
clusterFolders = sorted(clusterFolders)

In [31]:
for folder in clusterFolders:
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f if "clusterSize" in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        nucPath = join(nucleiPath, age, aorta)
        nucImage = fullNucImage16(nucPath)
        prop_dict = regionprops_table(nucImage, properties = ('label', 'centroid'))
        df = pd.DataFrame(prop_dict)
        clusters = np.load(join(folder,file), allow_pickle=True).item()
        dfNucLabels = pd.DataFrame()
        for index in clusters:
            dfHelp = pd.DataFrame({"label": clusters[index], "cluster": np.full(len(clusters[index]),index)})
            dfNucLabels = pd.concat([dfNucLabels, dfHelp], ignore_index= True)
        dfMerged = df.merge(dfNucLabels, on="label", how="left")
        dfMerged['cluster'] = dfMerged['cluster'].fillna(0)

        dfFiltered = dfMerged.loc[dfMerged["cluster"] > 0]
        dfFiltered = dfFiltered.reset_index(drop=True)
        dfFiltered = dfFiltered.reset_index(drop=False)
        nbrsNuc, distancesNuc, indicesNuc = kNN(dfFiltered.iloc[:,2:4], 17)

        distances = []
        for i in range(len(indicesNuc)):
            same = True
            j = 1
            while(same == True):
                if dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,0]]["cluster"].values != dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,j]]["cluster"].values:
                    distances.append(distancesNuc[i,j])
                    same = False
                else:
                    j += 1

        dfFiltered["1NNdifferentCluster"] = distances
        filename = "markedNN_noCellpose_" + age + "_" + aorta + ".csv"
        os.chdir(savePath)
        if not os.path.exists(join(savePath, age)):
            os.mkdir(age)
        os.chdir(join(savePath, age))
        if not os.path.exists(join(savePath, age, aorta)):
            os.mkdir(aorta)
        os.chdir(join(savePath,age, aorta))

        dfFiltered.to_csv(join(savePath, age, aorta, filename), index=False)

10-21
aorta01
10-21
aorta02
10-21
aorta03
10-21
aorta04
10-21
aorta05
10-30
aorta01
10-30
aorta02
10-30
aorta03
10-30
aorta04
10-30
aorta05
10-30
aorta06
10-30
aorta07
10-60
aorta01
10-60
aorta02
10-60
aorta03
10-60
aorta04
10-60
aorta05
10-60
aorta06
21-30
aorta01
21-30
aorta02
21-30
aorta03
21-30
aorta04
21-30
aorta05
30-60
aorta01
30-60
aorta02
30-60
aorta03
30-60
aorta04
30-60
aorta05
5-10
aorta01
5-10
aorta02
5-10
aorta03
5-10
aorta04
5-10
aorta06
5-10
aorta07
5-10
aorta08
5-30
aorta01
5-30
aorta02
5-30
aorta03
5-30
aorta04
5-30
aorta05
5-30
aorta06
5-60
aorta01
5-60
aorta02
5-60
aorta03


In [ ]:
#Nearest neighbour marked cells, cellPose

In [68]:
accuracy_value = 0.8

for folder in clusterFolders:
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f if "cellposeSegm" in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        print(age)
        print(aorta)
        nucPath = join(nucleiPath, age, aorta)
        nucImage = fullNucImage16(nucPath)
        prop_dict = regionprops_table(nucImage, properties = ('label', 'centroid'))
        df = pd.DataFrame(prop_dict)
        masks = np.load(join(folder,file), allow_pickle=True).item()["masks"]
        dfNucLabels = pd.DataFrame()
        for mask in np.unique(masks):
            if mask != 0:
                coord = np.where(mask == masks)
                segmentationNuc = np.unique(nucImage[coord])
                cluster = list()
                for j in segmentationNuc:
                    if j != 0:
                        #occurence of nucleus pixel in
                        nucOccur = np.bincount(nucImage[coord])[j]
                        #How many pixels does this nucleus have:
                        coordLen = len(np.where(nucImage == j)[0])
                        #check if most of nuclei values are in segmentation
                        if(nucOccur >= accuracy_value*coordLen):
                            cluster.append(j)

                if len(cluster) > 0:
                    dfHelp = pd.DataFrame({"label": cluster, "cluster": np.full(len(cluster),mask)})
                    dfNucLabels = pd.concat([dfNucLabels, dfHelp], ignore_index= True)

        dfMerged = df.merge(dfNucLabels, on="label", how="left")
        dfMerged['cluster'] = dfMerged['cluster'].fillna(0)

        dfFiltered = dfMerged.loc[dfMerged["cluster"] > 0]
        dfFiltered = dfFiltered.reset_index(drop=True)
        dfFiltered = dfFiltered.reset_index(drop=False)
        nbrsNuc, distancesNuc, indicesNuc = kNN(dfFiltered.iloc[:,2:4], 17)

        distances = []
        for i in range(len(indicesNuc)):
            same = True
            j = 1
            while(same == True):
                if dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,0]]["cluster"].values != dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,j]]["cluster"].values:
                    distances.append(distancesNuc[i,j])
                    same = False
                else:
                    j += 1

        dfFiltered["1NNdifferentCluster"] = distances
        filename = "markedNN_cellpose_" + age + "_" + aorta + ".csv"
        os.chdir(savePath)
        if not os.path.exists(join(savePath, age)):
            os.mkdir(age)
        os.chdir(join(savePath, age))
        if not os.path.exists(join(savePath, age, aorta)):
            os.mkdir(aorta)
        os.chdir(join(savePath,age, aorta))

        dfFiltered.to_csv(join(savePath, age, aorta, filename), index=False)

0-10
aorta01
0-10
aorta02
0-10
aorta03
0-10
aorta04
0-10
aorta05
0-10
aorta06
0-5
aorta01
0-5
aorta02
0-5
aorta03
0-5
aorta04
0-5
aorta05
0-5
aorta06
5-60
aorta04
5-60
aorta05
5-60
aorta06
5-60
aorta07
5-60
aorta08


In [43]:
np.unique(masks)

array([   0,   15,   16,   19,   24,   27,   28,   29,   31,   42,   44,
         45,   47,   52,   54,   55,   58,   64,   69,   70,   72,   73,
         75,   78,   89,   90,   94,   96,   98,  105,  108,  119,  121,
        129,  133,  136,  138,  144,  145,  151,  153,  155,  166,  168,
        169,  174,  178,  186,  188,  189,  204,  212,  217,  225,  235,
        236,  237,  240,  242,  246,  251,  256,  257,  264,  265,  266,
        277,  278,  279,  280,  288,  300,  304,  306,  307,  308,  312,
        317,  318,  321,  322,  324,  325,  329,  330,  331,  332,  339,
        341,  343,  347,  348,  350,  359,  363,  369,  370,  372,  373,
        375,  376,  378,  379,  381,  382,  384,  385,  386,  387,  388,
        389,  391,  392,  393,  398,  399,  400,  401,  405,  409,  414,
        417,  418,  425,  426,  427,  428,  432,  434,  437,  438,  441,
        442,  447,  449,  453,  456,  457,  458,  459,  461,  462,  463,
        466,  471,  473,  474,  479,  481,  484,  4

In [51]:
accuracy_value = 0.8
dfNucLabels = pd.DataFrame()
for mask in np.unique(masks):
    if mask != 0:
        coord = np.where(mask == masks)
        segmentationNuc = np.unique(nucImage[coord])
        cluster = list()
        for j in segmentationNuc:
            if j != 0:
                #occurence of nucleus pixel in
                nucOccur = np.bincount(nucImage[coord])[j]
                #How many pixels does this nucleus have:
                coordLen = len(np.where(nucImage == j)[0])
                #check if most of nuclei values are in segmentation
                if(nucOccur >= accuracy_value*coordLen):
                    cluster.append(j)

        if len(cluster) > 0:
            dfHelp = pd.DataFrame({"label": cluster, "cluster": np.full(len(cluster),mask)})
            dfNucLabels = pd.concat([dfNucLabels, dfHelp], ignore_index= True)

In [52]:
dfNucLabels

,label,cluster
0,10084,15
1,30025,16
2,10033,19
3,10062,19
4,20120,24
...,...,...
726,150936,1315
727,150969,1315
728,151019,1315
729,100816,1318


In [53]:
age

'5-60'

In [54]:
aorta

'aorta08'

In [62]:
dfMerged

,label,centroid-0,centroid-1,cluster
0,10001,1.800000,121.933333,0.0
1,10002,3.518519,178.703704,0.0
2,10003,2.461538,200.846154,0.0
3,10004,2.190476,264.571429,0.0
4,10005,2.157895,270.000000,0.0
...,...,...,...,...
13099,160725,1850.533333,1383.133333,0.0
13100,160726,1851.529412,1573.411765,0.0
13101,160727,1851.888889,1626.055556,0.0
13102,160728,1851.388889,1566.777778,0.0


In [60]:
dfMerged = df.merge(dfNucLabels, on="label", how="left")
dfMerged['cluster'] = dfMerged['cluster'].fillna(0)

In [63]:
dfFiltered = dfMerged.loc[dfMerged["cluster"] > 0]
dfFiltered = dfFiltered.reset_index(drop=True)
dfFiltered = dfFiltered.reset_index(drop=False)
nbrsNuc, distancesNuc, indicesNuc = kNN(dfFiltered.iloc[:,2:4], 17)

In [64]:
distances = []
for i in range(len(indicesNuc)):
    same = True
    j = 1
    while(same == True):
        if dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,0]]["cluster"].values != dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,j]]["cluster"].values:
            distances.append(distancesNuc[i,j])
            same = False
        else:
            j += 1

In [65]:
dfFiltered

,index,label,centroid-0,centroid-1,cluster
0,0,10033,16.666667,251.066667,19.0
1,1,10047,22.941176,164.882353,1189.0
2,2,10052,25.931034,326.379310,27.0
3,3,10062,32.607143,249.642857,19.0
4,4,10076,37.388889,30.833333,55.0
...,...,...,...,...,...
726,726,160589,1770.461538,1321.269231,1147.0
727,727,160607,1777.800000,1430.040000,1153.0
728,728,160621,1785.423077,1261.192308,1161.0
729,729,160638,1793.806452,1439.548387,1153.0


In [67]:
distances

[50.47453821744645,
 85.30987034751097,
 52.85873081873993,
 36.70262150856659,
 134.825353891417,
 48.44916480492501,
 83.96489134610007,
 36.70262150856659,
 50.82388810432371,
 32.87226284636655,
 21.447548029572836,
 21.447548029572836,
 25.500230626530097,
 18.77132860981815,
 12.000370364654966,
 14.193582274315983,
 49.87992402319968,
 41.75151452580241,
 32.71256978933668,
 73.12401781100053,
 32.71256978933668,
 25.08646853192636,
 25.08646853192636,
 38.491691859102374,
 51.62420306353257,
 13.137695908443481,
 13.137695908443481,
 56.526358901794104,
 39.696739300742486,
 30.443379251945355,
 23.6331464526306,
 38.14988398473769,
 17.225849497263326,
 35.25423176116158,
 35.25423176116158,
 17.225849497263326,
 42.11801546749021,
 40.200123638453775,
 53.84736546313336,
 25.343970192661892,
 30.110323060119146,
 20.606860616654092,
 20.606860616654092,
 16.829034799043058,
 42.172585980827016,
 61.75814495901489,
 16.829034799043058,
 38.13414373387466,
 53.51373822230425,
 

In [16]:
dfFiltered = dfMerged.loc[dfMerged["cluster"] > 0]
dfFiltered = dfFiltered.reset_index(drop=True)
dfFiltered = dfFiltered.reset_index(drop=False)
nbrsNuc, distancesNuc, indicesNuc = kNN(dfFiltered.iloc[:,2:4], 17)

In [17]:
distances = []
for i in range(len(indicesNuc)):
    same = True
    j = 1
    while(same == True):
        if dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,0]]["cluster"].values != dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,j]]["cluster"].values:
            distances.append(distancesNuc[i,j])
            same = False
        else:
            j += 1

In [21]:
distances

[27.4505883361359,
 14.372320619858158,
 14.372320619858158,
 32.502888989616174,
 29.315984990465633,
 28.705776880230523,
 28.705776880230523,
 60.68525517217918,
 61.585938663917965,
 65.88205378809822,
 50.23060107672911,
 50.23060107672911,
 17.02095765486305,
 16.156269908621333,
 17.157221327240766,
 16.156269908621333,
 16.225328305575843,
 55.11219134377937,
 76.71606761028485,
 55.11219134377937,
 29.315984990465633,
 76.68537097702307,
 57.74939291690843,
 30.368936280504254,
 25.76130939467937,
 42.95420817568404,
 17.010390796854946,
 17.010390796854946,
 50.13608041078718,
 36.20864896269725,
 42.95420817568404,
 27.0197494769995,
 27.0197494769995,
 31.755773733154207,
 28.60005074286284,
 29.647838556389882,
 28.60005074286284,
 19.50117735849164,
 18.731085133965372,
 18.731085133965372,
 42.34553953833705,
 89.0132265665093,
 67.36278281164269,
 40.772053314696244,
 44.392715056790315,
 34.482145532055085,
 28.108588490238713,
 17.479419326739606,
 55.6993126968641,
 

In [23]:
dfFiltered["distances"] = distances

In [24]:
dfFiltered

,index,label,centroid-0,centroid-1,cluster,distances
0,0,10028,17.280000,271.080000,17.0,27.450588
1,1,10056,32.800000,272.560000,17.0,14.372321
2,2,10070,41.500000,284.000000,20.0,14.372321
3,3,10234,143.576923,320.153846,52.0,32.502889
4,4,10263,163.789474,323.736842,52.0,29.315985
...,...,...,...,...,...,...
577,577,160707,1763.533333,1144.133333,716.0,31.281135
578,578,160710,1764.739130,1134.782609,716.0,39.152644
579,579,160726,1772.950000,1141.800000,716.0,37.440179
580,580,160736,1781.393939,1178.757576,712.0,37.909926


In [ ]:
savePath = "/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/nearestNeigbourNucleiComparison"
dfMerged.to_csv(join(savePath, "markedNN_noCellpose.csv"), index=False)

In [ ]:
nucleiPath = '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/nuclei'
clusterPath = '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red'

In [ ]:
clusterFolders = [x[0] for x in os.walk(clusterPath)]
clusterFolders = sorted(clusterFolders)

In [ ]:
for folder in clusterFolders:
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f if "cellposeSegm" in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        print(age)
        print(aorta)
        nucPath = join(nucleiPath, age, aorta)
        nucImage = fullNucImage16(nucPath)
        prop_dict = regionprops_table(nucImage, properties = ('label', 'centroid'))
        df = pd.DataFrame(prop_dict)
        clusters = np.load(join(folder,file), allow_pickle=True).item()
        dfNucLabels = pd.DataFrame()
        for index in clusters:
            dfHelp = pd.DataFrame({"label": clusters[index], "cluster": np.full(len(clusters[index]),index)})
            dfNucLabels = pd.concat([dfNucLabels, dfHelp], ignore_index= True)
        dfMerged = df.merge(dfNucLabels, on="label", how="left")
        dfMerged['cluster'] = dfMerged['cluster'].fillna(0)

In [ ]:
savePath = "/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/nearestNeigbourNucleiComparison"
dfMerged.to_csv(join(savePath, "markedNNCellpose.csv"), index=False)

In [87]:
dfNucDistancs.loc[(dfNucDistancs["age"] == "10-21") & (dfNucDistancs["aorta"] == "aorta01")]

,0,1,2,3,4,5,age,aorta
0,0.0,11.108746,13.196975,14.654392,21.832795,24.908592,10-21,aorta01
1,0.0,12.045796,16.176226,19.203140,19.700451,24.723267,10-21,aorta01
2,0.0,5.050201,6.731002,17.604388,18.870817,19.015413,10-21,aorta01
3,0.0,5.050201,8.921377,13.966375,15.604491,16.391696,10-21,aorta01
4,0.0,10.122191,11.416479,13.712593,17.989195,18.628452,10-21,aorta01
...,...,...,...,...,...,...,...,...
12427,0.0,9.083812,14.377495,18.743684,19.294231,21.319348,10-21,aorta01
12428,0.0,9.879728,15.329065,16.019129,22.128192,22.571276,10-21,aorta01
12429,0.0,9.094910,11.121016,11.160162,12.085586,17.944487,10-21,aorta01
12430,0.0,7.930138,9.052761,11.549145,16.914161,17.085431,10-21,aorta01


In [67]:
df

,label,centroid-0,centroid-1
0,10001,2.812500,116.000000
1,10002,3.347826,173.043478
2,10003,3.782609,192.739130
3,10004,4.481481,197.740741
4,10005,1.400000,221.600000
...,...,...,...
12427,160625,1487.625000,1143.875000
12428,160626,1487.950000,1169.600000
12429,160627,1488.150000,1060.450000
12430,160628,1488.227273,1103.909091


In [68]:
clusters = np.load(join(folder,file), allow_pickle=True).item()
clusters

{11: [30051],
 13: [40042],
 14: [10053],
 15: [10052],
 16: [20051],
 20: [40057],
 21: [20115],
 22: [30096],
 24: [30124, 30140, 30172],
 26: [40077],
 27: [20097],
 28: [30109, 30179],
 32: [10092],
 33: [40078],
 35: [10109],
 36: [20122],
 38: [20133],
 41: [10100],
 43: [30141],
 44: [40086],
 45: [40102],
 46: [40099],
 50: [40115],
 51: [20153],
 53: [30181, 30216],
 54: [40120],
 55: [30199],
 56: [40125, 40153],
 57: [30197],
 58: [40135],
 60: [30221],
 61: [10152],
 64: [20232],
 65: [20241],
 68: [30241],
 70: [10172],
 71: [30249],
 73: [20274],
 78: [20267],
 79: [20293, 20314, 20321],
 80: [40204],
 82: [40194],
 86: [30335],
 87: [20317, 20327, 20351, 20368, 20372],
 88: [30325, 30358, 30389],
 90: [30363, 30403],
 92: [20357],
 93: [20379],
 94: [30362],
 96: [30372, 30387],
 98: [40247],
 99: [40255],
 100: [20385],
 101: [20373],
 102: [30422],
 104: [10275],
 105: [10258],
 106: [40284],
 107: [30414],
 108: [40276],
 111: [30444, 30493],
 112: [40297, 40339],
 11

In [69]:
dfNucLabels = pd.DataFrame()
for index in clusters:
    dfHelp = pd.DataFrame({"label": clusters[index], "cluster": np.full(len(clusters[index]),index)})
    dfNucLabels = pd.concat([dfNucLabels, dfHelp], ignore_index= True)
dfNucLabels

,label,cluster
0,30051,11
1,40042,13
2,10053,14
3,10052,15
4,20051,16
...,...,...
853,100766,601
854,100772,601
855,70638,383
856,70687,383


In [70]:
dfMerged = df.merge(dfNucLabels, on="label", how="left")
dfMerged['cluster'] = dfMerged['cluster'].fillna(0)
dfMerged

,label,centroid-0,centroid-1,cluster
0,10001,2.812500,116.000000,0.0
1,10002,3.347826,173.043478,0.0
2,10003,3.782609,192.739130,0.0
3,10004,4.481481,197.740741,0.0
4,10005,1.400000,221.600000,0.0
...,...,...,...,...
12427,160625,1487.625000,1143.875000,0.0
12428,160626,1487.950000,1169.600000,0.0
12429,160627,1488.150000,1060.450000,0.0
12430,160628,1488.227273,1103.909091,0.0


In [71]:
dfFiltered = dfMerged.loc[dfMerged["cluster"] > 0]
dfFiltered = dfFiltered.reset_index(drop=True)
dfFiltered = dfFiltered.reset_index(drop=False)
dfFiltered

,index,label,centroid-0,centroid-1,cluster
0,0,10052,26.653846,311.307692,15.0
1,1,10053,27.846154,181.461538,14.0
2,2,10092,48.320000,240.000000,32.0
3,3,10100,49.458333,321.750000,41.0
4,4,10109,55.500000,175.500000,35.0
...,...,...,...,...,...
853,853,160564,1453.444444,1233.740741,810.0
854,854,160568,1457.029412,1196.676471,811.0
855,855,160579,1462.962963,1160.111111,823.0
856,856,160587,1467.000000,1061.080000,818.0


In [88]:
dfFiltered.iloc[:,1:3]

,label,centroid-0
0,10052,26.653846
1,10053,27.846154
2,10092,48.320000
3,10100,49.458333
4,10109,55.500000
...,...,...
853,160564,1453.444444
854,160568,1457.029412
855,160579,1462.962963
856,160587,1467.000000


In [89]:
nbrsNuc, distancesNuc, indicesNuc = kNN(dfFiltered.iloc[:,2:4], 17)

In [90]:
indicesNuc

array([[  0,   3,   2, ...,  11,  12,   8],
       [  1,   4,   5, ...,  14,  10,  11],
       [  2,   6,   1, ...,  13,  11,  14],
       ...,
       [855, 857, 851, ..., 842, 841, 852],
       [856, 804, 850, ..., 857, 846, 793],
       [857, 855, 854, ..., 852, 841, 856]])

In [35]:
if dfFiltered.loc[dfFiltered["index"] == indicesNuc[0,0]]["cluster"].values == dfFiltered.loc[dfFiltered["index"] == indicesNuc[0,0]]["cluster"].values:
    print("Yes")
else:
    print("False")

Yes


In [92]:
distances = []
for i in range(len(indicesNuc)):
    same = True
    j = 1
    while(same == True):
        if dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,0]]["cluster"].values != dfFiltered.loc[dfFiltered["index"] == indicesNuc[i,j]]["cluster"].values:
            distances.append(distancesNuc[i,j])
            same = False
        else:
            j += 1

In [91]:
distancesNuc

array([[  0.        ,  25.08159535,  74.52656711, ..., 161.62021147,
        168.41397154, 178.63528562],
       [  0.        ,  28.2891348 ,  50.28547752, ..., 211.04704956,
        216.44910166, 217.20055355],
       [  0.        ,  41.16022641,  62.0155614 , ..., 164.17560115,
        165.0061964 , 170.04118028],
       ...,
       [  0.        ,  18.33875715,  31.78424862, ..., 103.88782102,
        111.8232518 , 119.91167303],
       [  0.        ,  33.91955778,  56.05583288, ..., 116.92      ,
        118.30891712, 119.93615811],
       [  0.        ,  18.33875715,  21.17128204, ..., 102.57813777,
        114.5010917 , 116.92      ]])

In [93]:
np.median(distances)

27.36296433161492

In [78]:
dfFiltered

,index,label,centroid-0,centroid-1,cluster
0,0,10052,26.653846,311.307692,15.0
1,1,10053,27.846154,181.461538,14.0
2,2,10092,48.320000,240.000000,32.0
3,3,10100,49.458333,321.750000,41.0
4,4,10109,55.500000,175.500000,35.0
...,...,...,...,...,...
853,853,160564,1453.444444,1233.740741,810.0
854,854,160568,1457.029412,1196.676471,811.0
855,855,160579,1462.962963,1160.111111,823.0
856,856,160587,1467.000000,1061.080000,818.0


array([     0,  10001,  10002, ..., 160627, 160628, 160629], dtype=uint32)